In [ ]:
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.6/67.6 kB 755.0 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.8/413.8 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.9/138.9 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 3

In [ ]:
# Import necessary libraries and modules
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, pipeline
from datasets import load_dataset
import nltk

In [ ]:
!pip install transformers datasets

In [ ]:
#Set device
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Load the Flan-T5 tokenizer and model
tokenizer = T5Tokenizer.from_pretrained('csebuetnlp/mT5_multilingual_XLSum')
model = T5ForConditionalGeneration.from_pretrained('csebuetnlp/mT5_multilingual_XLSum')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/375 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/730 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
You are using a model of type mt5 to instantiate a model of type t5. This is not supported for all configurations of models and can yield errors.


pytorch_model.bin:   0%|          | 0.00/2.33G [00:00<?, ?B/s]

In [ ]:
# Define a function to generate prompts with examples
def generate_prompt(examples, new_text):
    prompt = "নীচে কিছু পাঠ্য সংক্ষিপ্তকরণের উদাহরণ দেওয়া হলো:\n\n"
    for example in examples:
        prompt += f"পাঠ্য: {example['text']}\nসংক্ষিপ্তসার: {example['summary']}\n\n"
    prompt += "---------------\n"
    prompt += "---------------\n"
    prompt += "এখন নিম্নলিখিত পাঠ্যটি সংক্ষিপ্ত করুন:\n"
    prompt += f"পাঠ্য: {new_text}\nসংক্ষিপ্তসার:"
    return prompt


In [ ]:
# Examples for the model to learn from
examples = [
    {
        "text": "করোনাভাইরাস মহামারীর কারণে বিশ্বব্যাপী অর্থনীতি বিপর্যস্ত হয়েছে। অনেক দেশ লকডাউন ঘোষণা করেছে এবং ব্যবসা প্রতিষ্ঠান বন্ধ হয়ে গেছে।",
        "summary": "করোনাভাইরাস মহামারীতে বিশ্ব অর্থনীতি বিপর্যস্ত, লকডাউনে ব্যবসা বন্ধ।"
    },
    {
        "text": "বাংলাদেশে সম্প্রতি বন্যা হয়েছে, যার ফলে হাজার হাজার মানুষ ঘরছাড়া হয়েছে। সরকারের পক্ষ থেকে ত্রাণ কার্যক্রম শুরু করা হয়েছে।",
        "summary": "বাংলাদেশে বন্যায় হাজার হাজার মানুষ ঘরছাড়া, ত্রাণ কার্যক্রম শুরু।"
    },
    {
        "text": "বাংলাদেশে নির্বাচন কমিশন নতুন ভোটার তালিকা প্রকাশ করেছে। এতে ১০ লাখ নতুন ভোটারের নাম অন্তর্ভুক্ত হয়েছে।",
        "summary": "নির্বাচন কমিশনের নতুন তালিকায় ১০ লাখ নতুন ভোটার।"
    }

]

In [ ]:
new_text = "শিশু-কিশোরদের নেশার আশঙ্কা দূর করতে টিকটকের মূল প্রতিষ্ঠান যথেষ্ট পদক্ষেপ না নিলে বৃহস্পতিবারই সেই কর্মসূচি বন্ধ করা হতে পারে৷ ডিজিটাল সার্ভিসেস অ্যাক্ট নামের আইনকে হাতিয়ার করে ইইউ ক্ষতিকর ও বেআইনি কনটেন্ট মোকাবিলার চেষ্টা করছে৷ টিকটক অবশ্য ইইউ'র হুমকি সম্পর্কে হতাশা প্রকাশ করে জানিয়েছে, ১৮ বছরের কমবয়সীদের জন্য রিওয়ার্ডস হাব বন্ধ রাখা হয়েছে৷ ফলে দুশ্চিন্তার কোনো কারণ থাকার কথা নয়৷ গত ফেব্রুয়ারি মাসে টিকটকের বিরুদ্ধে অন্য একটি তদন্ত শুরু হয়েছে ৷"

In [ ]:
# Generate the prompt
prompt = generate_prompt(examples, new_text)

In [ ]:
print(prompt)

নীচে কিছু পাঠ্য সংক্ষিপ্তকরণের উদাহরণ দেওয়া হলো:

পাঠ্য: করোনাভাইরাস মহামারীর কারণে বিশ্বব্যাপী অর্থনীতি বিপর্যস্ত হয়েছে। অনেক দেশ লকডাউন ঘোষণা করেছে এবং ব্যবসা প্রতিষ্ঠান বন্ধ হয়ে গেছে।
সংক্ষিপ্তসার: করোনাভাইরাস মহামারীতে বিশ্ব অর্থনীতি বিপর্যস্ত, লকডাউনে ব্যবসা বন্ধ।

পাঠ্য: বাংলাদেশে সম্প্রতি বন্যা হয়েছে, যার ফলে হাজার হাজার মানুষ ঘরছাড়া হয়েছে। সরকারের পক্ষ থেকে ত্রাণ কার্যক্রম শুরু করা হয়েছে।
সংক্ষিপ্তসার: বাংলাদেশে বন্যায় হাজার হাজার মানুষ ঘরছাড়া, ত্রাণ কার্যক্রম শুরু।

পাঠ্য: বাংলাদেশে নির্বাচন কমিশন নতুন ভোটার তালিকা প্রকাশ করেছে। এতে ১০ লাখ নতুন ভোটারের নাম অন্তর্ভুক্ত হয়েছে।
সংক্ষিপ্তসার: নির্বাচন কমিশনের নতুন তালিকায় ১০ লাখ নতুন ভোটার।

---------------
---------------
এখন নিম্নলিখিত পাঠ্যটি সংক্ষিপ্ত করুন:
পাঠ্য: শিশু-কিশোরদের নেশার আশঙ্কা দূর করতে টিকটকের মূল প্রতিষ্ঠান যথেষ্ট পদক্ষেপ না নিলে বৃহস্পতিবারই সেই কর্মসূচি বন্ধ করা হতে পারে৷ ডিজিটাল সার্ভিসেস অ্যাক্ট নামের আইনকে হাতিয়ার করে ইইউ ক্ষতিকর ও বেআইনি কনটেন্ট মোকাবিলার চেষ্টা করছে৷ টিকটক অবশ্য ইইউ'র হুমকি সম্

In [ ]:
# Tokenize the prompt
inputs = tokenizer(prompt, return_tensors="pt", max_length=512, truncation=True)

In [ ]:
print(inputs)

{'input_ids': tensor([[   259,  38928,  19011,   6173,  11692,  53512,   3198,   9039,  65078,
          35852,  90199,   1198, 113636,  63361,  22383,   7589,   5946,   2326,
           8508,    267,  53512,   3198,    267,   2212,  39693,  30544,  51507,
          36373,  12922,  13132,   3795,  44662,  31193,  84384, 113258,  29875,
          35602,   3154,  49681,  20207,   8137,    378,  10138,   9680,  18497,
            259, 118327,  10881, 156596,   6019,  40589,  31194,   2141,   3019,
           1244,   1807,   3170,  70641,   3532,   9504,  41061,    259,  37450,
          11421,  20694,   3019,    378,   9039,  65078,  35852,  29039,    267,
           2212,  39693,  30544,  51507,  36373,  12922,  61861,  31193,  29875,
          35602,   3154,  49681,  20207,    261,    259, 118327,  10881, 156596,
            994,  70641,   3532,    259,  37450,    378,  53512,   3198,    267,
          11977,    994,  17103, 107323,   2813,  65336,   8137,    261,    259,
          8748

In [ ]:
# Generate the summary
outputs = model.generate(input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"], max_length=150, num_beams=2, early_stopping=True)

# Decode the summary
summary = tokenizer.decode(outputs[0], skip_special_tokens=True)


In [ ]:
print("Generated Summary:", summary)

Generated Summary: টিকটকের বিরুদ্ধে ইউরোপীয় ইউনিয়নের হুমকি সম্পর্কে তদন্ত শুরু করার পর ইইউ'র সঙ্গে সংক্ষিপ্তকরণের বিষয়টি বেশ আলোচনায় এসেছে।


In [ ]:
!pip install nltk


In [ ]:
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

# Define the summaries
generated_summary = summary
reference_summary = "টিকটকের বিরুদ্ধে ইউরোপীয় ইউনিয়নের হুমকির পর তদন্ত শুরু হয়েছে এবং ইইউ'র পদক্ষেপ নিয়ে আলোচনা চলছে।"

# Tokenize the summaries
generated_tokens = generated_summary.split()
reference_tokens = reference_summary.split()

# Calculate BLEU score
smoothing_function = SmoothingFunction().method1
bleu_score = sentence_bleu([reference_tokens], generated_tokens, smoothing_function=smoothing_function)

print("Generated Summary:", generated_summary)
print("Reference Summary:", reference_summary)
print("BLEU Score:", bleu_score)


Generated Summary: টিকটকের বিরুদ্ধে ইউরোপীয় ইউনিয়নের হুমকি সম্পর্কে তদন্ত শুরু করার পর ইইউ'র সঙ্গে সংক্ষিপ্তকরণের বিষয়টি বেশ আলোচনায় এসেছে।
Reference Summary: টিকটকের বিরুদ্ধে ইউরোপীয় ইউনিয়নের হুমকির পর তদন্ত শুরু হয়েছে এবং ইইউ'র পদক্ষেপ নিয়ে আলোচনা চলছে।
BLEU Score: 0.18295654224495206


In [ ]:
from transformers import BartForConditionalGeneration, BartTokenizer
import torch

# Load the pre-trained BART model and tokenizer
model_name = 'facebook/bart-large-cnn'
model = BartForConditionalGeneration.from_pretrained(model_name)
tokenizer = BartTokenizer.from_pretrained(model_name)

# Define the summaries
generated_summary = "টিকটকের বিরুদ্ধে ইউরোপীয় ইউনিয়নের হুমকি সম্পর্কে তদন্ত শুরু করার পর ইইউ'র সঙ্গে সংক্ষিপ্তকরণের বিষয়টি বেশ আলোচনায় এসেছে।"
reference_summary = "টিকটকের বিরুদ্ধে ইউরোপীয় ইউনিয়নের হুমকির পর তদন্ত শুরু হয়েছে এবং ইইউ'র পদক্ষেপ নিয়ে আলোচনা চলছে।"

# Tokenize the inputs
def encode(text):
    return tokenizer(text, return_tensors='pt', max_length=1024, truncation=True, padding=True)

encoded_generated = encode(generated_summary)
encoded_reference = encode(reference_summary)

# Generate the BARTScore
def calculate_bartscore(model, tokenizer, src, tgt):
    with torch.no_grad():
        # Forward pass for the source to target
        output = model(**src, labels=tgt['input_ids'])
        # Calculate the log likelihood of tgt given src
        log_likelihood = output.loss.item()
    return log_likelihood

# Calculate BARTScore from generated to reference
bartscore_generated_to_reference = calculate_bartscore(model, tokenizer, encoded_generated, encoded_reference)
# Calculate BARTScore from reference to generated
bartscore_reference_to_generated = calculate_bartscore(model, tokenizer, encoded_reference, encoded_generated)

# Average the two scores for the final BARTScore
average_bartscore = (bartscore_generated_to_reference + bartscore_reference_to_generated) / 2

print("Generated Summary:", generated_summary)
print("Reference Summary:", reference_summary)
print("BARTScore Generated to Reference:", bartscore_generated_to_reference)
print("BARTScore Reference to Generated:", bartscore_reference_to_generated)
print("Average BARTScore:", average_bartscore)


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Generated Summary: টিকটকের বিরুদ্ধে ইউরোপীয় ইউনিয়নের হুমকি সম্পর্কে তদন্ত শুরু করার পর ইইউ'র সঙ্গে সংক্ষিপ্তকরণের বিষয়টি বেশ আলোচনায় এসেছে।
Reference Summary: টিকটকের বিরুদ্ধে ইউরোপীয় ইউনিয়নের হুমকির পর তদন্ত শুরু হয়েছে এবং ইইউ'র পদক্ষেপ নিয়ে আলোচনা চলছে।
BARTScore Generated to Reference: 2.1912941932678223
BARTScore Reference to Generated: 2.5691988468170166
Average BARTScore: 2.3802465200424194


In [ ]:
!pip install rouge

In [ ]:
from rouge import Rouge

# Define the summaries
generated_summary = "টিকটকের বিরুদ্ধে ইউরোপীয় ইউনিয়নের হুমকি সম্পর্কে তদন্ত শুরু করার পর ইইউ'র সঙ্গে সংক্ষিপ্তকরণের বিষয়টি বেশ আলোচনায় এসেছে।"
reference_summary = "টিকটকের বিরুদ্ধে ইউরোপীয় ইউনিয়নের হুমকির পর তদন্ত শুরু হয়েছে এবং ইইউ'র পদক্ষেপ নিয়ে আলোচনা চলছে।"

# Initialize the ROUGE scorer
rouge = Rouge()
# Extract the F1-scores for ROUGE-1, ROUGE-2, and ROUGE-L
rouge_1_f1 = scores['rouge-1']['f']
rouge_2_f1 = scores['rouge-2']['f']
rouge_l_f1 = scores['rouge-l']['f']


# Calculate the ROUGE scores
scores = rouge.get_scores(generated_summary, reference_summary, avg=True)
print("Generated Summary:", generated_summary)
print("Reference Summary:", reference_summary)
print("ROUGE scores:", scores)
print("ROUGE-1 F1 Score:", rouge_1_f1)
print("ROUGE-2 F1 Score:", rouge_2_f1)
print("ROUGE-L F1 Score:", rouge_l_f1)
print("BLEU Score:", bleu_score)


Generated Summary: টিকটকের বিরুদ্ধে ইউরোপীয় ইউনিয়নের হুমকি সম্পর্কে তদন্ত শুরু করার পর ইইউ'র সঙ্গে সংক্ষিপ্তকরণের বিষয়টি বেশ আলোচনায় এসেছে।
Reference Summary: টিকটকের বিরুদ্ধে ইউরোপীয় ইউনিয়নের হুমকির পর তদন্ত শুরু হয়েছে এবং ইইউ'র পদক্ষেপ নিয়ে আলোচনা চলছে।
ROUGE scores: {'rouge-1': {'r': 0.5333333333333333, 'p': 0.47058823529411764, 'f': 0.49999999501953135}, 'rouge-2': {'r': 0.2857142857142857, 'p': 0.25, 'f': 0.266666661688889}, 'rouge-l': {'r': 0.4666666666666667, 'p': 0.4117647058823529, 'f': 0.43749999501953124}}
ROUGE-1 F1 Score: 0.49999999501953135
ROUGE-2 F1 Score: 0.266666661688889
ROUGE-L F1 Score: 0.43749999501953124
BLEU Score: 0.18295654224495206


In [ ]:
from rouge import Rouge

# Define the summaries
generated_summary = "টিকটকের বিরুদ্ধে ইউরোপীয় ইউনিয়নের হুমকি সম্পর্কে তদন্ত শুরু করার পর ইইউ'র সঙ্গে সংক্ষিপ্তকরণের বিষয়টি বেশ আলোচনায় এসেছে।"
reference_summary = "টিকটকের বিরুদ্ধে ইউরোপীয় ইউনিয়নের হুমকির পর তদন্ত শুরু হয়েছে এবং ইইউ'র পদক্ষেপ নিয়ে আলোচনা চলছে।"

# Initialize the ROUGE scorer
rouge = Rouge()

# Calculate the ROUGE scores
scores = rouge.get_scores(generated_summary, reference_summary, avg=True)

# Extract the F1-scores for ROUGE-1, ROUGE-2, and ROUGE-L
rouge_1_f1 = scores['rouge-1']['f']
rouge_2_f1 = scores['rouge-2']['f']
rouge_l_f1 = scores['rouge-l']['f']

print("ROUGE-1 F1 Score:", rouge_1_f1)
print("ROUGE-2 F1 Score:", rouge_2_f1)
print("ROUGE-L F1 Score:", rouge_l_f1)


ROUGE-1 F1 Score: 0.49999999501953135
ROUGE-2 F1 Score: 0.266666661688889
ROUGE-L F1 Score: 0.43749999501953124
